# Medallion Architecture — Gold Layer

Business-ready aggregations and KPIs for dashboards, BI, and downstream ML.

| Gold Table | Grain | Purpose |
|---|---|---|
| `gold_daily_sales` | date × region × segment | Daily revenue, orders, profit |
| `gold_monthly_sales` | year-month × region × segment | Monthly roll-up with MoM/YoY growth |
| `gold_customer_rfm` | customer | RFM segmentation |
| `gold_product_performance` | brand × part_type | Revenue, margin, return rate |
| `gold_supplier_scorecard` | supplier | On-time %, delay, volume, profit |
| `gold_shipping_analysis` | ship_mode × region | Shipping mode performance |
| `gold_executive_summary` | year-quarter | Quarterly KPI summary |

**Prereq**: Run `02_silver_layer.ipynb` first.

## 1 — Configuration

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

CATALOG      = spark.catalog.currentCatalog()
SILVER       = f"{CATALOG}.retail_silver"
GOLD         = f"{CATALOG}.retail_gold"

print(f"Catalog : {CATALOG}")
print(f"Silver  : {SILVER}")
print(f"Gold    : {GOLD}")

## 2 — Create Gold Schema

In [ ]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {GOLD}")
spark.sql(f"COMMENT ON SCHEMA {GOLD} IS 'Gold layer — business aggregations, KPIs, and BI-ready tables'")
print(f"Schema ready: {GOLD}")

## 3 — Helpers

In [ ]:
def write_gold(df, table_name, cluster_cols=None, comment=None):
    fqn = f"{GOLD}.{table_name}"
    df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(fqn)
    if cluster_cols:
        spark.sql(f"ALTER TABLE {fqn} CLUSTER BY ({', '.join(cluster_cols)})")
    if comment:
        spark.sql(f"COMMENT ON TABLE {fqn} IS '{comment}'")
    cnt = spark.table(fqn).count()
    print(f"  ✓ {fqn:<55} {cnt:>12,} rows")
    return cnt

# Load Silver tables we'll reuse across multiple Gold tables
fact_li = spark.table(f"{SILVER}.fact_lineitem")
fact_ord = spark.table(f"{SILVER}.fact_orders")
dim_cust = spark.table(f"{SILVER}.dim_customer")

print("Gold helpers ready.")

---
## 4 — gold_daily_sales
Daily revenue, orders, and profit sliced by customer region and market segment.

In [ ]:
# Join lineitem with orders to get customer region/segment at the line level
li_with_order = (
    fact_li.alias("li")
    .join(
        fact_ord.select("order_key", "customer_region", "market_segment", "order_date").alias("o"),
        F.col("li.order_key") == F.col("o.order_key"),
        "inner"
    )
)

gold_daily_sales = (
    li_with_order
    .groupBy(
        F.col("o.order_date").alias("sale_date"),
        F.col("o.customer_region").alias("region"),
        F.col("o.market_segment").alias("segment"),
    )
    .agg(
        F.countDistinct("o.order_key").alias("num_orders"),
        F.sum("li.quantity").alias("total_quantity"),
        F.round(F.sum("li.net_revenue"), 2).alias("net_revenue"),
        F.round(F.sum("li.gross_revenue"), 2).alias("gross_revenue"),
        F.round(F.sum("li.supply_cost"), 2).alias("total_cost"),
        F.round(F.sum("li.profit"), 2).alias("total_profit"),
        F.round(F.avg("li.discount"), 4).alias("avg_discount"),
        F.countDistinct("li.part_key").alias("distinct_products"),
    )
    .withColumn("profit_margin_pct", F.round(F.col("total_profit") / F.col("net_revenue") * 100, 2))
)

write_gold(gold_daily_sales, "gold_daily_sales",
    cluster_cols=["sale_date", "region"],
    comment="Daily sales aggregation by region and market segment")
display(gold_daily_sales.orderBy("sale_date").limit(10))

## 5 — gold_monthly_sales
Monthly roll-up with month-over-month and year-over-year growth rates.

In [ ]:
daily = spark.table(f"{GOLD}.gold_daily_sales")

monthly_base = (
    daily
    .withColumn("year_month", F.date_format("sale_date", "yyyy-MM"))
    .withColumn("year",  F.year("sale_date"))
    .withColumn("month", F.month("sale_date"))
    .groupBy("year_month", "year", "month", "region", "segment")
    .agg(
        F.sum("num_orders").alias("num_orders"),
        F.sum("total_quantity").alias("total_quantity"),
        F.round(F.sum("net_revenue"), 2).alias("net_revenue"),
        F.round(F.sum("gross_revenue"), 2).alias("gross_revenue"),
        F.round(F.sum("total_cost"), 2).alias("total_cost"),
        F.round(F.sum("total_profit"), 2).alias("total_profit"),
        F.round(F.avg("avg_discount"), 4).alias("avg_discount"),
    )
)

# Window for MoM (partition by region+segment, order by year_month)
w_mom = Window.partitionBy("region", "segment").orderBy("year_month")
# Window for YoY (same month, previous year)
w_yoy = Window.partitionBy("region", "segment", "month").orderBy("year")

gold_monthly_sales = (
    monthly_base
    .withColumn("prev_month_revenue", F.lag("net_revenue", 1).over(w_mom))
    .withColumn("mom_growth_pct", F.round(
        (F.col("net_revenue") - F.col("prev_month_revenue")) / F.col("prev_month_revenue") * 100, 2))
    .withColumn("prev_year_revenue", F.lag("net_revenue", 1).over(w_yoy))
    .withColumn("yoy_growth_pct", F.round(
        (F.col("net_revenue") - F.col("prev_year_revenue")) / F.col("prev_year_revenue") * 100, 2))
    .withColumn("profit_margin_pct", F.round(F.col("total_profit") / F.col("net_revenue") * 100, 2))
    .drop("prev_month_revenue", "prev_year_revenue")
)

write_gold(gold_monthly_sales, "gold_monthly_sales",
    cluster_cols=["year_month", "region"],
    comment="Monthly sales with MoM and YoY growth rates by region and segment")
display(gold_monthly_sales.orderBy("year_month", "region").limit(15))

## 6 — gold_customer_rfm
RFM (Recency, Frequency, Monetary) segmentation — foundational for marketing analytics and AI/BI.

In [ ]:
# Reference date: latest order date in the dataset
max_date = fact_ord.agg(F.max("order_date")).collect()[0][0]

rfm_base = (
    fact_ord
    .groupBy("customer_key", "market_segment", "customer_nation", "customer_region")
    .agg(
        F.datediff(F.lit(max_date), F.max("order_date")).alias("recency_days"),
        F.countDistinct("order_key").alias("frequency"),
        F.round(F.sum("total_price"), 2).alias("monetary"),
        F.min("order_date").alias("first_order_date"),
        F.max("order_date").alias("last_order_date"),
        F.round(F.avg("total_price"), 2).alias("avg_order_value"),
    )
)

# Score each RFM dimension into quintiles (1=worst, 5=best)
# For recency, lower days = better, so we reverse
gold_customer_rfm = (
    rfm_base
    .withColumn("r_score", F.ntile(5).over(Window.orderBy(F.col("recency_days").desc())))
    .withColumn("f_score", F.ntile(5).over(Window.orderBy("frequency")))
    .withColumn("m_score", F.ntile(5).over(Window.orderBy("monetary")))
    .withColumn("rfm_score", F.col("r_score") + F.col("f_score") + F.col("m_score"))
    # Segment label
    .withColumn("rfm_segment",
        F.when(F.col("rfm_score") >= 13, "Champions")
         .when(F.col("rfm_score") >= 10, "Loyal Customers")
         .when(F.col("rfm_score") >= 8,  "Potential Loyalists")
         .when(F.col("rfm_score") >= 6,  "At Risk")
         .when(F.col("rfm_score") >= 4,  "Needs Attention")
         .otherwise("Lost")
    )
    .withColumn("customer_lifetime_days", F.datediff(F.col("last_order_date"), F.col("first_order_date")))
)

write_gold(gold_customer_rfm, "gold_customer_rfm",
    cluster_cols=["rfm_segment", "customer_region"],
    comment="Customer RFM segmentation with recency, frequency, monetary scores")
display(gold_customer_rfm.groupBy("rfm_segment").agg(
    F.count("*").alias("customers"),
    F.round(F.avg("monetary"), 0).alias("avg_monetary"),
    F.round(F.avg("frequency"), 1).alias("avg_frequency"),
    F.round(F.avg("recency_days"), 0).alias("avg_recency_days"),
).orderBy(F.desc("avg_monetary")))

## 7 — gold_product_performance
Product analytics by brand and type — revenue, margin, return rates.

In [ ]:
gold_product = (
    fact_li
    .groupBy("brand", "part_type", "price_band", "manufacturer")
    .agg(
        F.sum("quantity").alias("total_quantity_sold"),
        F.round(F.sum("net_revenue"), 2).alias("net_revenue"),
        F.round(F.sum("profit"), 2).alias("total_profit"),
        F.round(F.sum("supply_cost"), 2).alias("total_cost"),
        F.countDistinct("order_key").alias("num_orders"),
        F.round(F.avg("discount"), 4).alias("avg_discount"),
        # Return rate: fraction of lines with return_flag = 'R'
        F.round(
            F.sum(F.when(F.col("return_flag") == "R", 1).otherwise(0)) / F.count("*") * 100, 2
        ).alias("return_rate_pct"),
    )
    .withColumn("profit_margin_pct", F.round(F.col("total_profit") / F.col("net_revenue") * 100, 2))
    .withColumn("avg_revenue_per_order", F.round(F.col("net_revenue") / F.col("num_orders"), 2))
)

write_gold(gold_product, "gold_product_performance",
    cluster_cols=["brand"],
    comment="Product performance by brand and type — revenue, margin, return rate")
display(gold_product.orderBy(F.desc("net_revenue")).limit(15))

## 8 — gold_supplier_scorecard
Supplier quality and reliability metrics.

In [ ]:
gold_supplier = (
    fact_li
    .groupBy("supplier_key", "supplier_name", "supplier_nation", "supplier_region")
    .agg(
        F.count("*").alias("total_line_items"),
        F.countDistinct("order_key").alias("num_orders"),
        F.sum("quantity").alias("total_quantity"),
        F.round(F.sum("net_revenue"), 2).alias("net_revenue"),
        F.round(F.sum("profit"), 2).alias("total_profit"),
        F.round(F.avg("delivery_delay_days"), 1).alias("avg_delivery_delay_days"),
        F.round(F.avg("ship_delay_days"), 1).alias("avg_ship_delay_days"),
        # On-time: receipt_date <= commit_date (delay <= 0)
        F.round(
            F.sum(F.when(F.col("delivery_delay_days") <= 0, 1).otherwise(0)) / F.count("*") * 100, 2
        ).alias("on_time_delivery_pct"),
        F.round(
            F.sum(F.when(F.col("return_flag") == "R", 1).otherwise(0)) / F.count("*") * 100, 2
        ).alias("return_rate_pct"),
    )
    .withColumn("profit_margin_pct", F.round(F.col("total_profit") / F.col("net_revenue") * 100, 2))
)

write_gold(gold_supplier, "gold_supplier_scorecard",
    cluster_cols=["supplier_region"],
    comment="Supplier scorecard — on-time delivery, return rate, revenue, profit")
display(gold_supplier.orderBy(F.desc("net_revenue")).limit(15))

## 9 — gold_shipping_analysis
Shipping mode performance across regions.

In [ ]:
li_ship = fact_li.join(
    fact_ord.select("order_key", "customer_region"),
    "order_key", "inner"
)

gold_shipping = (
    li_ship
    .groupBy("ship_mode", "customer_region")
    .agg(
        F.count("*").alias("total_shipments"),
        F.sum("quantity").alias("total_quantity"),
        F.round(F.sum("net_revenue"), 2).alias("net_revenue"),
        F.round(F.avg("delivery_delay_days"), 1).alias("avg_delivery_delay_days"),
        F.round(F.avg("ship_delay_days"), 1).alias("avg_ship_delay_days"),
        F.round(
            F.sum(F.when(F.col("delivery_delay_days") <= 0, 1).otherwise(0)) / F.count("*") * 100, 2
        ).alias("on_time_pct"),
        F.round(
            F.sum(F.when(F.col("return_flag") == "R", 1).otherwise(0)) / F.count("*") * 100, 2
        ).alias("return_rate_pct"),
    )
)

write_gold(gold_shipping, "gold_shipping_analysis",
    comment="Shipping mode performance by customer region — delay, on-time rate, volume")
display(gold_shipping.orderBy("ship_mode", "customer_region"))

## 10 — gold_executive_summary
Quarterly C-level KPI roll-up.

In [ ]:
exec_base = (
    fact_ord
    .withColumn("year_quarter", F.concat(F.col("order_year"), F.lit("-Q"), F.col("order_quarter")))
    .groupBy("year_quarter", "order_year", "order_quarter")
    .agg(
        F.countDistinct("order_key").alias("total_orders"),
        F.countDistinct("customer_key").alias("active_customers"),
        F.round(F.sum("total_price"), 2).alias("gross_order_value"),
        F.round(F.avg("total_price"), 2).alias("avg_order_value"),
    )
)

# Add QoQ growth
w_q = Window.orderBy("year_quarter")
gold_exec = (
    exec_base
    .withColumn("prev_q_revenue", F.lag("gross_order_value", 1).over(w_q))
    .withColumn("qoq_revenue_growth_pct", F.round(
        (F.col("gross_order_value") - F.col("prev_q_revenue")) / F.col("prev_q_revenue") * 100, 2))
    .withColumn("prev_q_customers", F.lag("active_customers", 1).over(w_q))
    .withColumn("qoq_customer_growth_pct", F.round(
        (F.col("active_customers") - F.col("prev_q_customers")) / F.col("prev_q_customers") * 100, 2))
    .withColumn("revenue_per_customer", F.round(F.col("gross_order_value") / F.col("active_customers"), 2))
    .drop("prev_q_revenue", "prev_q_customers")
)

write_gold(gold_exec, "gold_executive_summary",
    comment="Quarterly executive KPI summary — orders, revenue, customer growth")
display(gold_exec.orderBy("year_quarter"))

## 11 — Gold Layer Summary

In [ ]:
gold_tables = [
    "gold_daily_sales", "gold_monthly_sales", "gold_customer_rfm",
    "gold_product_performance", "gold_supplier_scorecard",
    "gold_shipping_analysis", "gold_executive_summary",
]

print(f"{'Gold Table':<35} {'Rows':>12}  {'Columns':>8}")
print("=" * 60)
for t in gold_tables:
    df = spark.table(f"{GOLD}.{t}")
    print(f"{t:<35} {df.count():>12,}  {len(df.columns):>8}")

---
7 Gold tables in `retail_gold` — ready for SQL analytics (`04_gold_analytics.sql`), ML, and BI.

Continue with the SQL queries, then notebooks 05-09 for serving, ML, agents, dashboards, and the web app.